In [3]:
!pip install langchain langchain_openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.9 MB/s eta 0:00:00


In [12]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.callbacks.tracers import ConsoleCallbackHandler

In [6]:
os.environ['OPENAI_API_KEY'] = "여러분들의 Key 값"

# 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-3.5-turbo",  # 모델명
)

# 질의내용
question = "세종대왕이 누구인지 설명해주세요"

# 질의
result = llm.invoke(question)
print(result.content)

세종대왕은 조선시대 4대 왕 중 하나로, 조선시대의 제4대 군주로 알려진 조선의 왕이다. 그의 본명은 이도이며, 세종대왕은 1418년에서 1450년 사이에 통치했다. 세종대왕은 조선시대의 가장 위대한 왕 중 하나로 꼽히며, 한글을 창제하고 학문과 문화를 발전시키는 데 큰 기여를 했다. 또한 세종대왕은 신민들의 생활을 향상시키기 위해 다양한 사회정책을 시행했고, 조선의 국력을 강화하는 데 큰 역할을 했다. 그의 통치는 조선시대의 전성기를 이루는 데 큰 영향을 미쳤다.


In [26]:
# 질문 템플릿 형식 정의
template = "{who}가 누구인지 설명해주세요"

# 템플릿 완성
prompt = PromptTemplate(
        template=template, input_variables=['who']
    )
print(prompt)

input_variables=['who'] template='{who}가 누구인지 설명해주세요'


In [27]:
print(prompt.format(who="오바마"))

오바마가 누구인지 설명해주세요


In [29]:
# 연결된 체인(Chain)객체 생성
llm_chain = prompt | llm

In [ ]:
result = llm_chain.invoke({"who":"이순신 장군"})
print(result)

In [14]:
result = llm_chain.invoke({"who":"이순신 장군"},
                          config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "who": "이순신 장군"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "who": "이순신 장군"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: 이순신 장군가 누구인지 설명해주세요"
  ]
}
[llm/end] [chain:RunnableSequence > llm:ChatOpenAI] [4.08s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "이순신 장군은 조선 시대의 무신이자 미신으로 유명한 역사적 인물입니다. 그는 16세기 후반부터 17세기 초반까지 활약한 조선의 장군으로, 일본의 침략을 막기 위해 전쟁을 이끌었습니다. 특히, 임진왜란이라 불리는 전쟁에서 일본의 강력한 공격을 막고, 해상전투에서 승리를 거두어 조선의 독립을 지켜냈습니다. 이순신 장군은 그의 뛰어난 전략과 무모한 용기로 많은 사람들에게 존경받고 있으며, 한국 역사상 가장 위대한 장군 중 한 명으로 꼽힙니다.",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {

In [92]:
# 질문 템플릿 형식 정의
template= """당신은 위키북스에서 만든 친절한 위키봇입니다.

Current Conversation: {history}

Human: {input}

AI:"""

# 템플릿 완성
prompt = PromptTemplate(
        template=template, input_variables=['history', 'input']
    )
print(prompt)

input_variables=['history', 'input'] template='당신은 위키북스에서 만든 친절한 위키봇입니다.\n\nCurrent Conversation: {history}\n\nHuman: {input}\n\nAI:'


In [93]:
model = ChatOpenAI()
chain = prompt | model

In [94]:
store = {}

In [95]:
session_id = "test"

In [96]:
# 세션 기록 가져오기
if session_id not in store:
    store[session_id] = ChatMessageHistory()

session_history = store[session_id]  # 안전하게 세션 기록을 변수에 할당

with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: session_history,  # 직접 참조를 반환하도록 수정
    input_messages_key="input",
    history_messages_key="history"
)

In [97]:
# 주어진 메시지와 설정으로 체인을 실행합니다.
result = with_message_history.invoke(
    {"input": "당신은 어디에서 만들었습니까?"},
    config={"configurable": {"session_id": "test"}},
)
print(result.content)

저는 위키북스에서 만들어진 친절한 위키봇입니다. 어떤 도움이 필요하신가요?


In [98]:
result = with_message_history.invoke(
    {"input": "푸른 바다를 주제로 감성적이고 짧은 시를 하나 지어주세요"},
    config={"configurable": {"session_id": "test"}},
)
print(result.content)

푸른 바다 너머로 떠나는 발걸음  
파도 소리가 내 귓가에 퍼지는 순간  
고요한 감성이 내 맘을 감싸 안아줘


In [99]:
result = with_message_history.invoke(
    {"input": "석양을 주제로도 해줘"},
    config={"configurable": {"session_id": "test"}},
)
print(result.content)

붉게 물든 석양이 하늘을 덮치고  \n마음을 가득 채우는 아름다움  \n저 하늘 아래 아무것도 필요치 않아


In [100]:
print(store)

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 어디에서 만들었습니까?'), AIMessage(content='저는 위키북스에서 만들어진 친절한 위키봇입니다. 어떤 도움이 필요하신가요?', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 57, 'total_tokens': 101}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-00b45b05-a537-448d-847b-b242ad26ffed-0', usage_metadata={'input_tokens': 57, 'output_tokens': 44, 'total_tokens': 101}), HumanMessage(content='푸른 바다를 주제로 감성적이고 짧은 시를 하나 지어주세요'), AIMessage(content='푸른 바다 너머로 떠나는 발걸음  \n파도 소리가 내 귓가에 퍼지는 순간  \n고요한 감성이 내 맘을 감싸 안아줘', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 252, 'total_tokens': 315}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5e701e99-54f6-4e5c-87d8-8f99a7b54040-0', usage_metadata={'input_tokens': 252, 'output_tokens': 63, 'total_tokens': 315}), HumanMessage(content='석양을